# **UNet Grad-CAM for Infant Ultrasound Body Composition**

This notebook trains a UNet with the same number of epochs and settings as the main model, but is tailored for generating Grad-CAM explanations of FM/FFM regression. It uses preprocessed clinical ultrasound images (no PHI in-notebook), enforces 4D (N,C,H,W) inputs, and preserves decoder feature maps for Grad-CAM visualization. Training matches the standard setup (not a toy run), ensuring Grad-CAM outputs are representative of the full model performance.

The notebook contains Grad-CAM methods utilized in the paper titled: Enhancing Newborn Health Assessment: Ultrasound-based Body Composition Prediction Using Deep Learning Techniques in the journal Ultrasound in Medicine & Biology.

## The following cell defines all nessecary packages used to run the code.

Additionally, Google Drive is mounted to the notebook. The Drive contains ultrasound images of body regions corresponding to infants.  

In [ ]:
## Importing all nessecary packages and tools
import os
import cv2
import math
import torch
import random
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
from datetime import date
from skimage import io, color
from google.colab import drive
from torchvision import models
import skimage.morphology as mo
import matplotlib.pyplot as plt
import torch.nn.functional as F
from PIL import Image, ImageFilter
from scipy.ndimage import median_filter
from torch.utils.data import random_split
import torchvision.transforms.functional as TF
from torch.utils.data  import Dataset, DataLoader
from sklearn.model_selection import train_test_split


from torchvision import transforms
from torch.autograd import Variable
from torchvision.utils import save_image
# from utils import *
# import timm
# from timm.models.layers import DropPath, to_2tuple, trunc_normal_
# import types

drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Necessary Pre-Processing

The following four cells mount the proper folders and sets up the data and the patients with matching body composition metric. To do this, there must be another file, specifically a csv file which contains a nominal list of all patients, body composition metrics which can then be mapped to folders of corresponding patient_id.

The ultrasound data is not publically available.






**Step 1: Count total number of images across all patients in the dataset folder**


In [ ]:
# Root directory containing all patient subfolders with cropped ultrasound images
root_dir = '/content/gdrive/MyDrive/Ultrasound Files- Minnesota + Boston Collaboration/cropped_images_bluethingremoved'
# List of patient subdirectories (each folder corresponds to one patient)
patients = os.listdir(root_dir)
# Count total number of image files across all patients
total = 0
for patient in patients:
  total += len(os.listdir(os.path.join(root_dir, patient)))
print(total) # should be 721

721



**Step 2: Load patient metadata from CSV and compare with folder structure**

Here, we read csv data into a dataframe and collect all patient 'Study ID'. We print out number of patients in csv file and in folder. The number of patients in folder determine our dataset size for training and testing.

In [ ]:
# Read metadata file containing patient IDs and body composition measurements
df = pd.read_csv('/content/gdrive/MyDrive/Ultrasound Files- Minnesota + Boston Collaboration/Data_11.5.23_modified.csv')
# Extract Study IDs from the CSV
patients = df['Study_ID']
# Get list of patient folders in the cropped_images directory
folders = os.listdir('/content/gdrive/MyDrive/Ultrasound Files- Minnesota + Boston Collaboration/cropped_images') #cropped_images_bluethingremoved
#Confirm matching number of patients
print('Number of patients in csv file: ', len(patients))
print('Number of patients in folder: ', len(folders))

Number of patients in csv file:  65
Number of patients in folder:  65


**Step 3: Build dictionaries for labels (FM, FFM, Weight, Length)**

Here, create two dictionary, using 'Study ID' as key, and ['FM', 'FFM', 'Weight_visit', 'Length_visit'] as value. Create the test dictionary from the training dictionary.

In [ ]:
# Create dictionary mapping Study_ID -> [FM, FFM, Weight_visit, Length_visit]
labels_dict = df.set_index('Study_ID')[['FM', 'FFM', 'Weight_visit', 'Length_visit']].apply(list, axis=1).to_dict()
# Define which patient IDs will be reserved for testing
test_data = [17042418, 9021218, 35080318, 58041919, 65050619, 57032919, 50102518]#, , , 57032919, 67062119, 60042419, 44091318]
# Create test dictionary from training dictionary
test_labels_dict = {k: labels_dict[k] for k in test_data if k in labels_dict}
# Remove test patients from training dictionary
for i in test_data:
  del labels_dict[i]
PATIENTS = [str(num) for num in test_data]

print(len(labels_dict))
print(len(test_labels_dict))

58
7


**Step 4: The cell below checks that there's no missing images in the folder left.**

In [ ]:
# Assuming each patient in the dictionaries has a list of image paths as one of the values.
# If the dictionary structure is different, this code will need adjustment.

total_train_images = 0
for patient_id, data in labels_dict.items():

    patient_path = os.path.join(root_dir, str(patient_id))
    if os.path.exists(patient_path):
        total_train_images += len(os.listdir(patient_path))
    else:
        print(f"Warning: Directory not found for patient {patient_id}")


total_test_images = 0
for patient_id, data in test_labels_dict.items():
    patient_path = os.path.join(root_dir, str(patient_id))
    if os.path.exists(patient_path):
        total_test_images += len(os.listdir(patient_path))
    else:
        print(f"Warning: Directory not found for test patient {patient_id}")

print("Total number of images for training dataset:", total_train_images)
print("Total number of images for test dataset:", total_test_images)

Total number of images for training dataset: 581
Total number of images for test dataset: 75


##In the next following section, we define the Grad-CAM data-loading, training and statistics functions.

These functions are used primarily to split and process the ultrasound images into training and testing groups. The PatientDataset function and TestPatientDataset function are both used to process ultrasound images so that they are able to be thread into the deep learning models. Some modifications of the models are done in subsequent blocks due to shape differences in the first layer of the deep learning models.

The training function is also defined in this section. In this function, the loss function is defined, along with the optimizer. The data is parsed and assigned to be trained by a specific model which is indicated in the training function. The training function utilizes a data-loader, which then keeps track of the training and validation losses as the training begins.

Loss functions, and metric functions (including MAPE, MSE, RSME and MAE) are defined and instantiated in the sequence below too. These are used to quantify the performance of our models.

Patient Dataset (Grad-CAM Compatible)

In [ ]:
REGION_IDX = {"B": 0, "A": 1, "Q": 2}

class PatientDataset(Dataset):
    """
      PyTorch Dataset for loading cropped ultrasound images and associated patient labels
      for a single specified region (Biceps, Abdomen, or Quadriceps).

      This dataset handles:
      - Selecting valid patient folders from the cropped dataset directory
      - Excluding hold-out patients (if `test_data` global is defined)
      - Loading a single region image per patient (B, A, or Q)
      - Applying optional preprocessing (cropping, speckle reduction, thresholding, denoising)
      - Returning the requested target output (FM or FFM) along with weight and length

      Args:
          transform (callable, optional):
              Image transform/augmentation function applied after loading.
          augmented_dataset (bool):
              If True, apply augmentation to the *entire dataset* (not used in this override).
          augment (int):
              Level of augmentation (default 0 = none).
          threshold (bool):
              If True, apply pixel thresholding for noise reduction.
          speckle (bool):
              If True, apply median filtering for speckle noise reduction.
          despeckle (bool):
              If True, apply fast Non-Local Means denoising.
          region (str):
              Which region to load:
                  "B" = Biceps,
                  "A" = Abdomen,
                  "Q" = Quadriceps.
          number_of_image (int):
              Number of images to load per region (default = 1).
          crop (list of float):
              Fraction of each image to keep, indexed by region:
                  [Abdomen, Biceps, Quadriceps].
              Example: [0.8, 1, 0.5] keeps 80% of Abdomen, 100% of Biceps, 50% of Quadriceps.
          output (str):
              Target label to predict:
                  "FM" = Fat Mass,
                  "FFM" = Fat-Free Mass.

      Returns:
          image (torch.Tensor): Preprocessed ultrasound image tensor for the selected region.
          label (torch.Tensor): Target label (FM or FFM) as shape (1,).
          weight (torch.Tensor): Patient's body weight from metadata as shape (1,).
          length (torch.Tensor): Patient's body length from metadata as shape (1,).
    """
    def __init__(self, transform=None, augmented_dataset=False, augment=0, threshold=False, speckle=False,
                 despeckle=False, region="B", number_of_image=1, crop=[1, 1, 1], output="FM"):
        self.root_dir = '/content/gdrive/MyDrive/Ultrasound Files- Minnesota + Boston Collaboration/cropped_images'
        self.transform = transform
        self.augment = augment
        self.threshold = threshold
        self.speckle = speckle
        self.augmented_dataset = augmented_dataset
        self.despeckle = despeckle
        self.region = region
        self.number_of_image = number_of_image
        self.crop = crop
        self.output = output

        # Build a clean list of valid patient IDs up front
        all_ids = [p for p in os.listdir(self.root_dir) if p.isdigit()]
        # Optionally exclude hold-out IDs
        holdout = set(map(str, test_data)) if 'test_data' in globals() else set()
        candidates = [pid for pid in all_ids if pid not in holdout]

        self.patients = []
        for pid in candidates:
            path = os.path.join(self.root_dir, pid)
            if any(f"_{self.region}" in fn for fn in os.listdir(path)):
                self.patients.append(pid)

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        patient_id = self.patients[idx]
        patient_path = os.path.join(self.root_dir, patient_id)

        # image tensor (3, H, W)
        image = self.load_patient_data(patient_path)

        # label/aux as float32 tensors (shape (1,))
        label  = torch.tensor(self.get_label(int(patient_id)), dtype=torch.float32).view(1)
        weight = torch.tensor(labels_dict[int(patient_id)][2], dtype=torch.float32).view(1)
        length = torch.tensor(labels_dict[int(patient_id)][3], dtype=torch.float32).view(1)

        return image, label, weight, length

    def get_label(self, patient_id):
        if self.output in ("FM", "PerFM"):
            return labels_dict[patient_id][0]
        if self.output in ("FFM", "PerFFM"):
            return labels_dict[patient_id][1]
        raise ValueError(f"Invalid output type: {self.output}")

    def preprocess(self, img_path, crop_frac):
        image = Image.open(img_path).convert("RGB")
        w, h = image.size
        image = image.crop((0, 0, w, round(h * float(crop_frac))))
        if self.speckle:
            image = image.filter(ImageFilter.MedianFilter(size=5))
        if self.threshold:
            arr = np.array(image)
            arr = np.where(arr < 100, 0, arr)
            image = Image.fromarray(arr)
        if self.despeckle:
            arr = np.array(image)
            arr = cv2.fastNlMeansDenoisingColored(arr, None, h=10, templateWindowSize=7, searchWindowSize=21)
            image = Image.fromarray(arr)
        if self.transform:
            image = self.transform(image)  # -> Tensor (3,H,W)
        return image

    def load_patient_data(self, patient_path):
        r = self.region
        crop_idx = REGION_IDX.get(r, 0)
        for img_file in os.listdir(patient_path):
            if f"_{r}" in img_file:
                img_path = os.path.join(patient_path, img_file)
                return self.preprocess(img_path, self.crop[crop_idx])
        raise ValueError(f"No image found for region {r} in {patient_path}")

Test Patient Dataset (Grad-CAM Compatible)

In [ ]:
class TestPatientDataset(Dataset):
    """
    PyTorch Dataset for loading cropped ultrasound images and labels for a fixed
    set of test patients.

    This dataset handles:
    - Selecting patient folders from a provided test patient list (`PATIENTS`)
    - Loading multiple regions per patient (Abdomen, Biceps, Quadriceps) depending on `region_combination`
    - Applying optional preprocessing (cropping, speckle reduction, thresholding, denoising)
    - Applying optional data augmentation (horizontal/vertical flips, rotations) based on `augment` level
    - Returning the requested target output (FM or FFM, absolute or percentage) along with weight and length

    Args:
        transform (callable, optional):
            Image transform/augmentation function applied after loading.
        augmented_dataset (bool):
            If True, apply augmentation to the *entire dataset* instead of per-image.
        augment (int):
            Level of augmentation:
                0 = none,
                5 = flip + rotation augmentations,
                8 = stronger flips/rotations,
                15 = extensive rotation set.
        threshold (bool):
            If True, apply pixel thresholding for noise reduction.
        speckle (bool):
            If True, apply median filtering for speckle noise reduction.
        despeckle (bool):
            If True, apply fast Non-Local Means denoising.
        region_combination (str):
            Which regions to include:
                "B" = Biceps,
                "A" = Abdomen,
                "Q" = Quadriceps,
                "BA", "BQ", "AQ", "BAQ" = multiple regions.
        number_of_image (int):
            Number of images to load per region (1, 2, or 3).
        crop (list of float):
            Fraction of each image to keep for [Abdomen, Biceps, Quadriceps].
            Example: [0.8, 1, 0.5] keeps 80% of Abdomen, full Biceps, 50% Quadriceps.
        output (str):
            Target label to predict:
                "FM"    = Fat Mass,
                "FFM"   = Fat-Free Mass,
                "PerFM" = Percent Fat Mass,
                "PerFFM"= Percent Fat-Free Mass.

    Returns:
        images (list[torch.Tensor]): List of preprocessed ultrasound images for the patient.
        label (float): Target label (FM, FFM, or percentage).
        weight (float): Patient's body weight from metadata.
        length (float): Patient's body length from metadata.
    """

    def __init__(self, transform=None, augmented_dataset=False, augment=0, threshold=False, speckle=False,
                 despeckle=False, region_combination='A', number_of_image=1, crop=[1, 1, 1], output="FM"):
        self.root_dir = '/content/gdrive/MyDrive/Ultrasound Files- Minnesota + Boston Collaboration/cropped_images'
        self.patients = PATIENTS
        self.transform = transform
        self.augment = augment
        self.threshold = threshold
        self.speckle = speckle
        self.augmented_dataset = augmented_dataset
        self.despeckle = despeckle
        self.region_combination = region_combination
        self.number_of_image = number_of_image
        # Crop in the order of Abdomen, Biceps, and Quadriceps
        self.crop = crop
        self.output = output

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        patient_id = self.patients[idx]
        patient_path = os.path.join(self.root_dir, patient_id)
        images = self.load_patient_data(patient_path)
        if self.output == "FM":
            return images, test_labels_dict[int(patient_id)][0], test_labels_dict[int(patient_id)][2], test_labels_dict[int(patient_id)][3]
        elif self.output == "FFM":
            return images, test_labels_dict[int(patient_id)][1], test_labels_dict[int(patient_id)][2], test_labels_dict[int(patient_id)][3]
        elif self.output == "PerFM":
            return images, test_labels_dict[int(patient_id)][0], test_labels_dict[int(patient_id)][2], test_labels_dict[int(patient_id)][3]
        elif self.output == "PerFFM":
            return images, test_labels_dict[int(patient_id)][1], test_labels_dict[int(patient_id)][2], test_labels_dict[int(patient_id)][3]
        else:
            raise ValueError(f"Invalid output type: {self.output}")

    def preprocess(self, img_path, MF_size, images, threshold, h, templateWindowSize, searchWindowSize, crop):
        image = Image.open(img_path)
        width, height = image.size
        crop_rectangle = (0, 0, width, round(height * crop))
        image = image.crop(crop_rectangle)
        if self.speckle:
            image = image.filter(ImageFilter.MedianFilter(size=MF_size))
        if self.threshold:
            image = torch.where(image < threshold / 255, torch.tensor(0.0), image)
        if self.despeckle:
            image_np = np.array(image)
            despeckled_image = cv2.fastNlMeansDenoisingColored(image_np, None, h=h, templateWindowSize=templateWindowSize, searchWindowSize=searchWindowSize)
            image = Image.fromarray(despeckled_image)
        if not self.augmented_dataset:
            image = self.transform(image)
        images.append(image)

        # Augmentation
        if self.augment >= 5:
            imx = image
            imx = TF.hflip(imx)
            images.append(imx)
            imx = TF.vflip(imx)
            images.append(imx)
            angle = random.choice([-30, -90, -60, -45, -15, 0, 15, 30, 45, 60, 90])
            imx = TF.rotate(imx, angle)
            images.append(imx)
            angle = random.choice([-30, -90, -60, -45, -15, 0, 15, 30, 45, 60, 90])
            imx = TF.rotate(imx, angle)
            images.append(imx)
        if self.augment >= 8:
            imx = TF.vflip(imx)
            images.append(imx)
            angle = random.choice([15])
            imx = TF.rotate(imx, angle)
            images.append(imx)
            angle = random.choice([45])
            imx = TF.rotate(imx, angle)
            images.append(imx)
        if self.augment >= 15:
            imx = image
            angle = random.choice([-30, -90, -60, -45, -15, 0, 15, 30, 45, 60, 90])
            imx = TF.rotate(imx, angle)
            images.append(imx)
            imx = image
            angle = random.choice([-90])
            imx = TF.rotate(imx, angle)
            images.append(imx)
            imx = image
            angle = random.choice([-60])
            imx = TF.rotate(imx, angle)
            images.append(imx)
            angle = random.choice([-45])
            imx = TF.rotate(imx, angle)
            images.append(imx)
            angle = random.choice([-15])
            imx = TF.rotate(imx, angle)
            images.append(imx)
            angle = random.choice([30])
            imx = TF.rotate(imx, angle)
            images.append(imx)
            angle = random.choice([60])
            imx = TF.rotate(imx, angle)
            images.append(imx)
        return images

    def load_patient_data(self, patient_path):
        images = []
        threshold = 100
        AB = 0
        BICEP = 0
        QUAD = 0
        MF_size = 5
        h = 10
        templateWindowSize = 7
        searchWindowSize = 21
        for img_file in os.listdir(patient_path):
            # Process Abdomen images
            if 'A' in self.region_combination and '_A' in img_file and AB < self.number_of_image:
                img_path = os.path.join(patient_path, img_file)
                images = self.preprocess(img_path, MF_size, images, threshold, h, templateWindowSize, searchWindowSize, self.crop[0])
                AB += 1
            # Process Bicep images
            elif 'B' in self.region_combination and '_B' in img_file and BICEP < self.number_of_image:
                img_path = os.path.join(patient_path, img_file)
                images = self.preprocess(img_path, MF_size, images, threshold, h, templateWindowSize, searchWindowSize, self.crop[1])
                BICEP += 1
            # Process Quad images
            elif 'Q' in self.region_combination and '_Q' in img_file and QUAD < self.number_of_image:
                img_path = os.path.join(patient_path, img_file)
                images = self.preprocess(img_path, MF_size, images, threshold, h, templateWindowSize, searchWindowSize, self.crop[2])
                QUAD += 1
        return images

In [ ]:
def im_converterX(tensor):
  """
    Convert a PyTorch tensor into a NumPy image (H x W x C) for visualization.

    Steps:
        1. Move tensor to CPU (if on GPU).
        2. Detach from computation graph (so gradients are not tracked).
        3. Convert to NumPy array.
        4. Rearrange dimensions from (C, H, W) -> (H, W, C).
        5. Scale pixel values into [0, 1] range and clip.

    Args:
        tensor (torch.Tensor):
            Image tensor in CHW format, typically with values normalized
            to [0, 1] or [-1, 1].

    Returns:
        numpy.ndarray:
            Image in HWC format with pixel values clipped between 0 and 1.
    """
  image = tensor.cpu().clone().detach().numpy()
  image = image.transpose(1,2,0)
  image = image * np.array((1, 1, 1))
  image = image.clip(0, 1)
  return image

In [ ]:
class MAPELoss(nn.Module):
    """
    Mean Absolute Percentage Error (MAPE) Loss for regression tasks.

    This loss is useful when the scale of the target values varies a lot,
    because it expresses error as a percentage of the true value.

    Formula:
        MAPE = mean( |(target - pred) / (target + epsilon)| ) * 100

    Args:
        epsilon (float, optional): Small constant to avoid division by zero
                                   when target values are close to 0. Default = 1e-7.

    Forward Args:
        pred (torch.Tensor): Predicted values from the model.
        target (torch.Tensor): Ground truth values.

    Returns:
        torch.Tensor: Scalar tensor containing the MAPE loss (percentage).
    """
    def __init__(self, epsilon=1e-7):
        super(MAPELoss, self).__init__()
        self.epsilon = epsilon

    def forward(self, pred, target):
        return torch.mean(torch.abs((target - pred) / (target + self.epsilon))) * 100

In [ ]:
class CustomLoss(nn.Module):
    """
    Custom regression loss that combines Mean Squared Error (MSE) and
    Mean Absolute Error (L1/MAE) using weighted averaging.

    Motivation:
        - MSE penalizes large errors more strongly (sensitive to outliers).
        - L1 (MAE) is more robust to outliers but less sensitive to small differences.
        - Combining both gives a balanced trade-off.

    Formula:
        total_loss = weight_mse * MSE(input, target)
                   + weight_l1  * L1(input, target)

    Args:
        weight_mse (float): Weight for the MSE component. Default = 0.6
        weight_l1 (float): Weight for the L1 component.  Default = 0.4

    Forward Args:
        input (torch.Tensor): Predicted values from the model.
        target (torch.Tensor): Ground truth values.

    Returns:
        torch.Tensor: Scalar tensor containing the weighted loss.
    """

    def __init__(self, weight_mse=0.6, weight_l1=0.4):
        super(CustomLoss, self).__init__()
        self.weight_mse = weight_mse
        self.weight_l1 = weight_l1
        self.mse_loss = nn.MSELoss()
        self.l1_loss = nn.L1Loss()

    def forward(self, input, target):
        mse_loss = self.mse_loss(input, target)
        l1_loss = self.l1_loss(input, target)
        total_loss = self.weight_mse * mse_loss + self.weight_l1 * l1_loss
        return total_loss

In [ ]:
def custom_loss(output, target):
    mse_loss = nn.MSELoss()(output, target)
    penalty = torch.mean(F.relu(-output))  # Penalize negative predictions
    return mse_loss + penalty

The Grad-CAM training function trains and validates the model for the same number of epochs as the main pipeline, while enforcing 4D (N,C,H,W) inputs and preserving gradients needed for CAM.
It supports MSE, MAE, and MAPE losses, works with both UNet and EffNet variants, and can optionally incorporate patient metadata (weight, length) during the forward pass.
Optimization uses Adam with optional ExponentialLR scheduling; the loop runs on GPU, logs batch/epoch losses, and returns train/validation loss histories for reproducible CAM generation.

In [ ]:
def training(epochs, criterion, lr, model, MODEL, adaptive=True, weight_length=False):
    """
      Train and validate a Grad-CAM compatible model using selected loss function,
      optimizer, and optional adaptive learning rate scheduling. Ensures 4D inputs
      and preserves gradients for CAM visualization.

      Args:
          epochs (int): Number of training epochs.
          criterion (str): Loss function to use. Options:
              - "MSE"  -> Mean Squared Error
              - "MAE"  -> Mean Absolute Error
              - "MAPE" -> Mean Absolute Percentage Error
          lr (float): Learning rate for optimizer.
          model (nn.Module): PyTorch model to be trained.
          MODEL (str): Model type identifier ("EffNet_LP", "EffNet_FT", "UNet").
          adaptive (bool, optional): If True, applies exponential LR decay (gamma=0.95).
                                    Default = True.
          weight_length (bool, optional): If True, includes patient metadata (weight & length)
                                          in the forward pass. Default = False.

      Returns:
          tuple: (train_running_loss_history, validation_running_loss_history)
              - train_running_loss_history (list[float]): Avg. training loss per epoch.
              - validation_running_loss_history (list[float]): Avg. validation loss per epoch.
    """
    if criterion == "MSE":
        criterion = nn.MSELoss()
    elif criterion == "MAE":
        criterion = nn.L1Loss()
    elif criterion == "MAPE":
        criterion = MAPELoss()

    # Move the model to GPU
    model = model.to('cuda')

    ### Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    if adaptive:
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

    train_running_loss_history = []
    validation_running_loss_history = []

    for e in range(epochs):
        train_running_loss = 0.0
        validation_running_loss = 0.0
        model.train()
        for ith_batch, batch in enumerate(train_loader):
            # Ensure X_train is properly shaped
            if isinstance(batch[0], torch.Tensor):
                X_train = batch[0].to('cuda').float()
            else:
                X_train = torch.stack([img for img in batch[0]]).to('cuda').float()

            # Remove extra dimensions if present
            if X_train.dim() == 5:  # If it's a 5D tensor, squeeze out unnecessary dimension
                X_train = X_train.squeeze(1)

            if weight_length:
                if MODEL == "EffNet_LP" or MODEL == "EffNet_FT":
                    additional_data_train = torch.tensor([[batch[2].item(), batch[3].item()]]).to('cuda').float()
                elif MODEL == "UNet":
                    weight = batch[2].to('cuda').float()[0]
                    length = batch[3].to('cuda').float()[0]
                y_train = batch[1].to('cuda').float()[0]
                y_pred = model(X_train, weight, length)[0]
            else:
                y_train = batch[1].to('cuda').float()[0]
                y_pred = model(X_train)[0]

            # Compute loss
            loss = criterion(y_pred, y_train)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if ith_batch % 5 == 0:
                print('Epoch: ', e + 1, 'Batch: ', ith_batch, 'Current Loss: ', loss.item())

            train_running_loss += loss.item()

        with torch.no_grad():
            model.eval()
            for ith_batch, batch in enumerate(test_loader):
                # Ensure X_val is properly shaped
                if isinstance(batch[0], torch.Tensor):
                    X_val = batch[0].to('cuda').float()
                else:
                    X_val = torch.stack([img for img in batch[0]]).to('cuda').float()

                # Remove extra dimensions if present
                if X_val.dim() == 5:  # If it's a 5D tensor, squeeze out unnecessary dimension
                    X_val = X_val.squeeze(1)

                if weight_length:
                    if MODEL == "EffNet_LP" or MODEL == "EffNet_FT":
                        additional_data_val = torch.tensor([[batch[2].item(), batch[3].item()]]).to('cuda').float()
                    elif MODEL == "UNet":
                        weight = batch[2].to('cuda').float()[0]
                        length = batch[3].to('cuda').float()[0]
                    y_val = batch[1].to('cuda').float()[0]
                    y_out = model(X_val, weight, length)[0]
                else:
                    y_val = batch[1].to('cuda').float()[0]
                    y_out = model(X_val)[0]

                val_loss = criterion(y_out, y_val)
                validation_running_loss += val_loss.item()

            print("================================================================================")
            print("Epoch {} completed".format(e + 1))
            train_epoch_loss = train_running_loss / len(train_loader)
            validation_epoch_loss = validation_running_loss / len(test_loader)
            print("Average train loss is {}: ".format(train_epoch_loss))
            print("Average validation loss is {}".format(validation_epoch_loss))
            print("================================================================================")
            train_running_loss_history.append(train_epoch_loss)
            validation_running_loss_history.append(validation_epoch_loss)

        torch.cuda.empty_cache()

        if adaptive:
            scheduler.step()

    return train_running_loss_history, validation_running_loss_history

The last cell defines our statistics functions which will be used extensively later on to quantify our model truths.

In [ ]:
def mean_absolute_error(y_true, y_pred):
    return torch.mean(torch.abs(y_true - y_pred))
def mean_squared_error(y_true, y_pred):
    return torch.mean((y_true - y_pred)**2)
def root_mean_squared_error(y_true, y_pred):
    return math.sqrt(torch.mean((y_true - y_pred)**2))
def mean_absolute_percentage_error(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)) * 100
def r_squared(y_true, y_pred):
    ss_residual = torch.sum((y_true - y_pred)**2)
    ss_total = torch.sum((y_true - torch.mean(y_true))**2)
    r2 = 1 - (ss_residual / ss_total)
    return r2

### Data Loader (Grad-CAM)

We standardize inputs and enforce 4D (N,C,H,W) tensors for CAM. Images are resized to 256×256, converted to tensors, and normalized with (0.5, 0.5, 0.5). We instantiate PatientDataset with no augmentation/denoising, a single region "Q" (Quadriceps), number_of_image=1, crop=[1,1,1], and output="FM". Any None samples are filtered out. The dataset is split 90/10 via random_split into train/test; both loaders use batch_size=1 and shuffle=True. A one-batch sanity check unpacks (images, label, weight, length) and, if needed, stacks the image list to ensure a 4D tensor (expected shape (1, 3, 256, 256)), confirming labels/auxiliary metadata are correctly retrieved for training and later Grad-CAM visualization.

In [ ]:
# Define normalization factors
normalization_factors = (0.5, 0.5, 0.5)

# Define transformation
tx_X = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(normalization_factors, normalization_factors)
])

# Create the dataset with the required parameters
dataset = PatientDataset(
    transform=tx_X,
    augmented_dataset=False,
    augment=0,
    threshold=False,
    speckle=False,
    despeckle=False,
    region="Q",  # Choose one region "B", "A", or "Q"
    number_of_image=1,       # Only one image per region
    crop=[1, 1, 1],          # Crop percentages for each region
    output="FM"              # Output label type "FM" or "FFM"
)

dataset = [item for item in dataset if item is not None]

# Split the dataset into training and testing sets
train_size = int(0.9 * len(dataset))  # 90% for training
test_size = len(dataset) - train_size  # Remaining 10% for testing
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
batch_size = 1
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=True)

# Print the number of batches in each DataLoader
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of testing batches: {len(test_loader)}")

# Fetch a single batch from the train loader to verify
first_batch = next(iter(train_loader))

# Unpack the batch
first_batch_images, label_FM, weight, length = first_batch

# Convert the list to a tensor if necessary
if isinstance(first_batch_images, list):
    first_batch_images = torch.stack(first_batch_images)

# Print the shapes to verify
print(f"Shape of first batch images: {first_batch_images.shape}")  # Expected: (1, 3, 256, 256)
print(f"Label FM: {label_FM}")
print(f"Weight: {weight}")
print(f"Length: {length}")

Number of training batches: 52
Number of testing batches: 6
Shape of first batch images: torch.Size([1, 3, 256, 256])
Label FM: tensor([[0.4044]])
Weight: tensor([[2.5007]])
Length: tensor([[46.]])


## UNet Model with 4D Compatibility for Grad-CAM

The following section defines the UNet architecture adapted for Grad-CAM visualization in body composition regression.
Unlike a standard segmentation UNet, this implementation preserves a 4D feature map for CAM extraction, applies dropout for regularization, and includes fully connected layers to reduce the output map to a scalar prediction (FM or FFM).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv(nn.Module):
    '''performs two convolution with batch normalization and LeakyReLU activation'''
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.LeakyReLU(0.1, inplace=False),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.LeakyReLU(0.1, inplace=False),
        )

    def forward(self, x):
        x = self.conv(x)
        return x

class InConv(nn.Module):
    '''input convolution block to raw images'''
    def __init__(self, in_ch, out_ch):
        super(InConv, self).__init__()
        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x

class Down(nn.Module):
    '''downsampling with maxpooling and double_conv'''
    def __init__(self, in_ch, out_ch):
        super(Down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x

class Up(nn.Module):
    '''upsampling and skip connections'''
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(Up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch // 2, in_ch // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diff1 = x2.size()[2] - x1.size()[2]
        diff2 = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, (diff1 // 2, diff1 - diff1 // 2, diff2 // 2, diff2 - diff2 // 2))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x

class OutConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x

class UNet(nn.Module):
    ''' the Unet class and architecture'''
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        '''Encoder path, contracting'''
        self.inc = InConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        '''Decoder path, expanding'''
        self.up1 = Up(1024, 256)
        self.up2 = Up(512, 128)
        self.up3 = Up(256, 64)
        self.up4 = Up(128, 64)
        '''output and regression layers'''
        self.outc = OutConv(64, n_classes)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(65536, 1)
        self.linear2 = nn.Linear(1, 1)

    def forward(self, x):
        '''forward pass of the Unet model above'''
        x = x.float()
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        x = torch.sigmoid(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

In [ ]:
## Instantiated the model and sending to GPU
MODEL = "UNet"
model = UNet(3, 1).cuda()

## Model Training

In [ ]:
torch.cuda.empty_cache()
CRITERION = "MSE" # other options include MAE, MAPE, Custom
WL = False # Include weight + length metadata (True/False)
EPOCHS = 90
LR = 0.001  #0.001 for UNet
ADAPTIVE = True # Use adaptive learning rate scheduling
train_history, validation_history = training(EPOCHS, CRITERION, LR, model, MODEL, ADAPTIVE, WL)

Epoch:  1 Batch:  0 Current Loss:  0.14244669675827026
Epoch:  1 Batch:  5 Current Loss:  222.01121520996094
Epoch:  1 Batch:  10 Current Loss:  69.27972412109375
Epoch:  1 Batch:  15 Current Loss:  8.507641792297363
Epoch:  1 Batch:  20 Current Loss:  8.213659286499023
Epoch:  1 Batch:  25 Current Loss:  3.5066912174224854
Epoch:  1 Batch:  30 Current Loss:  0.04494277387857437
Epoch:  1 Batch:  35 Current Loss:  0.13024316728115082
Epoch:  1 Batch:  40 Current Loss:  1.7445549964904785
Epoch:  1 Batch:  45 Current Loss:  0.6128560304641724
Epoch:  1 Batch:  50 Current Loss:  0.5151197910308838
Epoch 1 completed
Average train loss is 36.00118534802683: 
Average validation loss is 0.28266832903803635
Epoch:  2 Batch:  0 Current Loss:  3.8470723628997803
Epoch:  2 Batch:  5 Current Loss:  1.368618130683899
Epoch:  2 Batch:  10 Current Loss:  0.5557239651679993
Epoch:  2 Batch:  15 Current Loss:  0.3017197549343109
Epoch:  2 Batch:  20 Current Loss:  1.260148286819458
Epoch:  2 Batch:  2

### Load Test Data (Grad-CAM, UNet)

In [ ]:
# Grad-CAM testing
normalization_factors = 0.5
tx_X = transforms.Compose([transforms.Resize((256, 256)),
                           transforms.ToTensor(),
                           transforms.Normalize(normalization_factors, normalization_factors)])
REGION = "Q"
OUTPUT = "FM"
test_dataset = TestPatientDataset(transform = tx_X,
                 augmented_dataset = False,
                 augment = 0,
                 threshold = False,
                 speckle = True,
                 despeckle = False,
                 region_combination = REGION,
                 number_of_image = 2,
                 crop = [1, 1, 1],
                 output = OUTPUT)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print(len(test_loader))

true_labels = []
predictions = []
WL = False

model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    for ith_batch, batch in enumerate(test_loader):
        # Convert the list of images to a single tensor and move to GPU
        # Check if batch[0] is a list of tensors and stack them correctly
        if isinstance(batch[0], list):
            X_train = torch.stack(batch[0]).to('cuda')
        else:
            X_train = batch[0].to('cuda')

        y_train = batch[1].to('cuda').float()  # Move the batch of labels to GPU and convert to float

        # Ensure X_train has the correct shape: [batch_size, channels, height, width]
        if X_train.dim() == 5:
            # If the tensor has an extra dimension, remove it
            X_train = X_train.view(-1, *X_train.shape[2:])

        if WL:
            if MODEL == "EffNet_LP" or MODEL == "EffNet_FT":
                additional_data_train = torch.tensor([[batch[2].item(), batch[3].item()]]).to('cuda').float()
                y_pred = model(X_train, additional_data_train)
            elif MODEL == "UNet":
                weight = batch[2].to('cuda').float()
                length = batch[3].to('cuda').float()
                y_pred = model(X_train, weight, length)
        else:
            y_pred = model(X_train)

        # Append true labels and predictions
        true_labels.append(y_train)
        predictions.append(y_pred)

# Convert lists to tensors
true_labels = torch.cat(true_labels)
predictions = torch.cat(predictions)

# If necessary, move predictions and labels back to CPU
true_labels = true_labels.cpu().numpy()
predictions = predictions.cpu().numpy()

mae = np.mean(np.abs(predictions - true_labels))
mse = np.mean((predictions - true_labels) ** 2)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((true_labels - predictions) / true_labels)) * 100

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}")

print("\nFirst five true labels and predictions:")
for i in range(min(5, len(true_labels))):
    print(f"True label: {true_labels[i]}, Prediction: {predictions[i]}")

7
MAE: 0.11215952783823013
MSE: 0.019884878769516945
RMSE: 0.1410137563943863
MAPE: 67.25102996826172

First five true labels and predictions:
True label: 0.18799999356269836, Prediction: [0.23783615]
True label: 0.11249999701976776, Prediction: [0.10962443]
True label: 0.15880000591278076, Prediction: [0.1736444]
True label: 0.1216999962925911, Prediction: [0.19718358]
True label: 0.22089999914169312, Prediction: [0.04468233]


### Grad-CAM
The following section implements Grad-CAM for UNet.
GradCAM registers forward/backward hooks on a chosen conv layer to capture activations (B,C,H,W) and gradients, then computes a heatmap by global-averaging the grads to get channel weights and forming a weighted sum over activations, ReLU, and [0–1] normalization. apply_grad_cam preprocesses a single image (256×256, normalized), generates the Grad-CAM, overlays it on the RGB image, and saves to {out_dir}/{layer}/{body_part}/. run_gradcam_for_body_part iterates all patient folders, matches images by body-part pattern (A/B/Q), applies Grad-CAM, and writes outputs—providing a reproducible pipeline to visualize which regions drive the scalar FM/FFM prediction.

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import cv2
from torchvision import transforms
from PIL import Image

class GradCAM:
    """
      Generate and apply Grad-CAM visualizations for a trained deep learning model.

      The GradCAM class registers forward and backward hooks on a chosen layer
      to capture feature map activations and gradients during backpropagation.
      It then computes a heatmap highlighting important regions that contribute
      to the model’s scalar regression output (FM or FFM).

      The apply_grad_cam function preprocesses a single image, generates the CAM
      from the specified layer, overlays the heatmap onto the original image, and
      saves the result to disk.

      Args:
          model (nn.Module): Trained PyTorch model for which Grad-CAM is generated.
          layer_name (str): Name of the convolutional layer to target for CAM.
          image_path (str): Path to the input image.
          body_part (str): Region label (e.g., "A" for Abdomen, "B" for Biceps, "Q" for Quadriceps).
          out_dir (str): Directory where Grad-CAM overlay images are saved.

      Returns:
          np.ndarray (GradCAM.generate): Normalized heatmap (H, W) in [0,1].
          str (apply_grad_cam): File path to the saved Grad-CAM overlay image.
  """
    def __init__(self, model, layer_name: str):
        self.model = model
        self.model.eval()
        # Resolve the real module instance from the string
        try:
            self.target_layer = self.model.get_submodule(layer_name)
        except AttributeError:
            # For older PyTorch, fall back to manual walk
            mod = self.model
            for part in layer_name.split('.'):
                mod = getattr(mod, part)
            self.target_layer = mod

        self.gradients = None
        self.activations = None

        # Keep handles so they don't get GC'd
        self._fwd_handle = self.target_layer.register_forward_hook(self._forward_hook)
        self._bwd_handle = self.target_layer.register_full_backward_hook(self._backward_hook)

    def _forward_hook(self, module, inputs, output):
        # Save feature maps (B, C, H, W)
        self.activations = output.detach()

    def _backward_hook(self, module, grad_input, grad_output):
        # grad_output is a tuple; the first element is dL/d(activations)
        self.gradients = grad_output[0].detach()

    def remove_hooks(self):
        self._fwd_handle.remove()
        self._bwd_handle.remove()

    @torch.no_grad()
    def _normalize_cam(self, cam):
        cam -= cam.min()
        denom = cam.max() + 1e-8
        cam /= denom
        return cam

    def generate(self, input_tensor):
        """
        input_tensor: (1, C, H, W) on same device as model
        Returns: np.ndarray heatmap (H, W) in [0,1]
        """
        # Forward
        self.model.zero_grad(set_to_none=True)
        output = self.model(input_tensor)        # regression: shape (1, 1) or (1,)
        # Backward: for regression, push gradient of 1 through the scalar
        if output.dim() == 2:        # (B,1)
            grad_target = torch.ones_like(output)
        else:                        # (B,)
            grad_target = torch.ones_like(output).unsqueeze(-1)
        output.backward(grad_target, retain_graph=False)

        # Fetch grads & acts
        grads = self.gradients      # (1, C, H, W)
        acts  = self.activations    # (1, C, H, W)
        if grads is None or acts is None:
            raise RuntimeError("Hooks did not capture gradients/activations. "
                               "Ensure the chosen layer is a conv layer and "
                               "no .detach() is called before backward.")

        # Global average pooling over spatial dims -> weights (1, C, 1, 1)
        weights = grads.mean(dim=(2, 3), keepdim=True)
        # Weighted combination
        cam = (weights * acts).sum(dim=1, keepdim=False)    # (1, H, W)
        cam = F.relu(cam)[0].cpu().numpy()                  # (H, W)
        cam = self._normalize_cam(cam)
        return cam

def apply_grad_cam(image_path, model, layer_name, body_part, out_dir):
    device = next(model.parameters()).device
    image = Image.open(image_path).convert("RGB")

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
    ])
    x = transform(image).unsqueeze(0).to(device)  # (1, C, H, W)

    cam_engine = GradCAM(model, layer_name)
    try:
        cam = cam_engine.generate(x)  # (H, W) in [0,1]
    finally:
        cam_engine.remove_hooks()

    # Overlay
    img_np = np.array(image.resize((256,256)))
    heatmap = (cam * 255.0).astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(img_np, 0.5, heatmap, 0.5, 0)

    # Save
    os.makedirs(os.path.join(out_dir, layer_name, body_part), exist_ok=True)
    out_path = os.path.join(out_dir, layer_name, body_part,
                            f"Grad-CAM_{os.path.basename(image_path)}")
    cv2.imwrite(out_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
    return out_path

In [ ]:
import os

# Map flexible user input → filename pattern + folder name
_BODY_PART_MAP = {
    "A":   ("_A",   "A"),
    "ABD": ("_A",   "A"),
    "ABDOMEN": ("_A","A"),
    "B":   ("_B",   "B"),
    "BICEP": ("_B", "B"),
    "Q":   ("_Q",   "Q"),
    "QUAD":("_Q",   "Q"),
    "QUADRICEPS": ("_Q","Q"),
}

def run_gradcam_for_body_part(model, layer_name, body_part, patients,
                              root_dir="/content/gdrive/MyDrive/Ultrasound Files- Minnesota + Boston Collaboration/cropped_images",
                              out_dir="/content/gdrive/MyDrive/Ultrasound Files- Minnesota + Boston Collaboration/Grad-CAM/just_testing_code_2"):
    """
    Runs Grad-CAM only on the given list of test patients.

    Args:
        model (nn.Module): Trained model to explain.
        layer_name (str): Layer name to target for CAM (e.g., "outc").
        body_part (str): Region ("A", "B", "Q").
        patients (list[str]): List of patient IDs to process (e.g., PATIENTS).
        root_dir (str): Root directory of cropped_images.
        out_dir (str): Directory where Grad-CAM results are saved.
    """
    # Normalize body_part
    key = str(body_part).strip().upper()
    if key not in _BODY_PART_MAP:
        raise ValueError(f"Unknown body_part={body_part}. Use one of: {list(_BODY_PART_MAP.keys())}")
    pattern, part_folder = _BODY_PART_MAP[key]

    save_dir = os.path.join(out_dir, layer_name, part_folder)
    os.makedirs(save_dir, exist_ok=True)

    # uncomment below code if you want to run grad-cam on training data as well as testing data
    #patients = [p for p in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, p))]
    #patients.sort()

    processed, errors = 0, 0
    for pid in patients:
        pdir = os.path.join(root_dir, pid)
        # Find all images for this body part (in case there are multiple)
        files = [f for f in os.listdir(pdir) if pattern in f]
        if not files:
            continue
        for fname in files:
            img_path = os.path.join(pdir, fname)
            try:
                out_path = apply_grad_cam(
                    image_path=img_path,
                    model=model,
                    layer_name=layer_name,
                    body_part=part_folder,
                    out_dir=out_dir
                )
                print(f"[OK] {pid} → {os.path.basename(out_path)}")
                processed += 1
            except Exception as e:
                print(f"[ERR] {pid} / {fname}: {e}")
                errors += 1

    print(f"\nDone. Processed: {processed}, Errors: {errors}, Saved to: {save_dir}")

# usage:
run_gradcam_for_body_part(model, layer_name="outc", body_part="Q", patients=PATIENTS)

[OK] 17042418 → Grad-CAM_17042418_QUAD1_R.jpg
[OK] 17042418 → Grad-CAM_17042418_QUAD3_R.jpg
[OK] 17042418 → Grad-CAM_17042418_QUAD4_R.jpg
[OK] 17042418 → Grad-CAM_17042418_QUAD2_R.jpg
[OK] 17042418 → Grad-CAM_17042418_QUAD5_R.jpg
[OK] 9021218 → Grad-CAM_9021218_QUAD1_R.jpg
[OK] 9021218 → Grad-CAM_9021218_QUAD2_R.jpg
[OK] 9021218 → Grad-CAM_9021218_QUAD4_R.jpg
[OK] 9021218 → Grad-CAM_9021218_QUAD3_R.jpg
[OK] 35080318 → Grad-CAM_35080318_QUAD4_R.jpg
[OK] 35080318 → Grad-CAM_35080318_QUAD1_R.jpg
[OK] 35080318 → Grad-CAM_35080318_QUAD3_R.jpg
[OK] 35080318 → Grad-CAM_35080318_QUAD2_R.jpg
[OK] 58041919 → Grad-CAM_58041919_QUAD3_R.jpg
[OK] 58041919 → Grad-CAM_58041919_QUAD1_R.jpg
[OK] 58041919 → Grad-CAM_58041919_QUAD2_R.jpg
[OK] 65050619 → Grad-CAM_65050619_QUAD2_R.jpg
[OK] 65050619 → Grad-CAM_65050619_QUAD3_R.jpg
[OK] 65050619 → Grad-CAM_65050619_QUAD1_R.jpg
[OK] 57032919 → Grad-CAM_57032919_QUAD2_R.jpg
[OK] 57032919 → Grad-CAM_57032919_QUAD5_R.jpg
[OK] 57032919 → Grad-CAM_57032919_QUAD4_R.